In [1]:
import pickle

import pandas as pd
pd.set_option('display.max_columns', 100)
pd.options.mode.copy_on_write = True

import csv
import time

import numpy as np
from geopy.geocoders import Nominatim
# import unittest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from tqdm.notebook import tqdm as tqdmn

In [2]:
# # # Производим десериализацию и извлекаем модель из файла формата pkl
with open('data/df_1.pkl', 'rb') as pkl_file:
    df = pickle.load(pkl_file)

# df = pd.read_csv('data/df_1.csv',
#                  low_memory=False)
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,PrivatePool,target,city_is_capital,city_is_million,city_is_big,city_is_large,claster,distance_capital,distance_million
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,418000.0,False,False,False,False,-1.0,100.137884,655.199337
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,310000.0,False,False,False,True,0.0,436.589228,1519.395159
2,for sale,NaN,townhouse,1624 S Newkirk St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,NaN,209000.0,False,True,False,False,1.0,149.770696,11.871086
3,Active,NaN,Florida,552 Casanova Ct,NaN,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",NaN,Poinciana,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,NaN,181500.0,False,False,False,False,NaN,NaN,NaN
4,Active,NaN,NaN,6094 Mingle Dr,NaN,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,NaN,68000.0,False,False,True,False,2.0,305.956266,681.212051


In [3]:
df = df[['city', 'state', 'zipcode']]
df = df.replace('',np.NaN)
df = df.dropna().drop_duplicates()
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5524 entries, 0 to 5523
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city     5524 non-null   object
 1   state    5524 non-null   object
 2   zipcode  5524 non-null   object
dtypes: object(3)
memory usage: 129.6+ KB


In [4]:
df['zipcode'] = df['zipcode'] .astype('str')
df['Full_Address'] = df['city'].str.cat( df[['state', 'zipcode']], sep=' ') + ' ' + 'USA'
df['Full_Address']

0       Southern Pines NC 28387 USA
1       Spokane Valley WA 99216 USA
2         Philadelphia PA 19145 USA
3            Poinciana FL 34759 USA
4              Memphis TN 38115 USA
                   ...             
5519           Midwood NY 11210 USA
5520       Cooper City FL 33026 USA
5521        Petersburg FL 33701 USA
5522            Canton OH 44704 USA
5523            Queens NY 11428 USA
Name: Full_Address, Length: 5524, dtype: object

In [5]:
df.Full_Address = df.Full_Address.str.replace('/', ' ')

In [6]:
df['Url'] = ['https://www.google.com/maps/search/' + i for i in df['Full_Address'] ]

In [8]:
Url_With_Coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(#"chromedriver.exe",
                          options=option,
                           )

In [9]:
for url in tqdmn(df.Url, leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element(By.CSS_SELECTOR,'meta[itemprop=image]').get_attribute('content'))


    time.sleep(0.2)


driver.close()

  0%|          | 0/5524 [00:00<?, ?it/s]

In [10]:
with open('Url_With_Coordinates.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates)

In [7]:
with open('Url_With_Coordinates.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        Url_With_Coordinates = i
        break

In [8]:
len(Url_With_Coordinates)

5524

In [9]:
Url_With_Coordinates[:5]

['https://maps.google.com/maps/api/staticmap?center=35.162964%2C-79.369696&zoom=12&size=900x900&language=en&sensor=false&client=google-maps-frontend&signature=5JB09i27JV22rGx79ZTfyEmNnc8',
 'https://maps.google.com/maps/api/staticmap?center=47.6714505%2C-117.192152&zoom=13&size=900x900&language=en&sensor=false&client=google-maps-frontend&signature=29qQ7NTt8I0gJOgyKbyXwMr0zs0',
 'https://maps.google.com/maps/api/staticmap?center=39.91177345%2C-75.1915324&zoom=13&size=900x900&language=en&sensor=false&client=google-maps-frontend&signature=boYtzXLqXKEQvIAqYmhd8V57VpY',
 'https://maps.google.com/maps/api/staticmap?center=28.08453%2C-81.4390798&zoom=12&size=900x900&language=en&sensor=false&client=google-maps-frontend&signature=L73_BC7V0dQGCNQ5b4-BpBXArZU',
 'https://maps.google.com/maps/api/staticmap?center=35.0595875%2C-89.8678739&zoom=14&size=900x900&language=en&sensor=false&client=google-maps-frontend&signature=ARt-5bTsCDIum8fvxBeI-08tuuM']

In [10]:
df['Url_With_Coordinates'] = Url_With_Coordinates

In [11]:
df = df[df.Url_With_Coordinates.str.contains('&zoom=')]
df = df[df.Url_With_Coordinates.str.contains('center=')]

In [12]:
df['Url_With_Coordinates'][0]

'https://maps.google.com/maps/api/staticmap?center=35.162964%2C-79.369696&zoom=12&size=900x900&language=en&sensor=false&client=google-maps-frontend&signature=5JB09i27JV22rGx79ZTfyEmNnc8'

In [13]:
df['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in df['Url_With_Coordinates'] ]
df['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in df['Url_With_Coordinates'] ]

In [14]:
df = df.drop('Url_With_Coordinates',axis=1)

In [15]:
clustering = GaussianMixture(215)
df['claster'] = clustering.fit_predict(df[['lat','long']])

In [16]:
# сохранение
# Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\model_GaussianMixture.pkl', 'wb') as output:
    pickle.dump(clustering, output)

In [21]:
df = df.drop(['Full_Address','Url'],axis=1)
df

,city,state,zipcode,lat,long,claster
0,Southern Pines,NC,28387,35.162964,-79.369696,197
1,Spokane Valley,WA,99216,47.6714505,-117.192152,29
2,Philadelphia,PA,19145,39.91177345,-75.1915324,47
3,Poinciana,FL,34759,28.08453,-81.4390798,67
4,Memphis,TN,38115,35.0595875,-89.8678739,35
...,...,...,...,...,...,...
5519,Midwood,NY,11210,40.6256315,-73.94579,131
5520,Cooper City,FL,33026,26.02974415,-80.26129795,137
5521,Petersburg,FL,33701,27.82638305,-82.6974574,55
5522,Canton,OH,44704,40.7998363,-81.345395,152


In [22]:
# сохранение
# Производим сериализацию и записываем результат в файл формата pkl
with open('data/city_center.pkl', 'wb') as output:
    pickle.dump(df, output)


df.to_csv('data/city_center.csv', index_label=False)